In [70]:
import sys
import pathlib
pth=str(pathlib.Path().absolute())
sys.path.append(('\\').join(pth.split('\\')[:-1])+"\\Utils")
from Utilities import *
from MF import *
from Visualization_utilities import *
from transformation import Transformation
from Signatures import Signature
import os

In [71]:
def return_rot(vect, ang):
    cosA = np.cos(ang)
    sinA = np.sin(ang)
    oneMinusCosA = 1-cosA
    out = np.zeros((3, 3))          
    out[0, 0] = (cosA + vect[0] * vect[0] * oneMinusCosA)
    out[0, 1] = (vect[1] * vect[0] * oneMinusCosA + vect[2] * sinA)
    out[0, 2] = (vect[2] * vect[0] * oneMinusCosA - vect[1] * sinA)
    out[1, 0] = (vect[0] * vect[1] * oneMinusCosA - vect[2] * sinA)
    out[1, 1] = (cosA + vect[1] * vect[1] * oneMinusCosA)
    out[1, 2] = (vect[2] * vect[1] * oneMinusCosA + vect[0] * sinA)
    out[2, 0] = (vect[0] * vect[2] * oneMinusCosA + vect[1] * sinA)
    out[2, 1] = (vect[1] * vect[2] * oneMinusCosA - vect[0] * sinA)
    out[2, 2] = (cosA + vect[2] * vect[2] * oneMinusCosA)
    return out

In [94]:
import open3d as o3d

pts = 'Beam-W_W8x2_pulg_ac_210.pts'
#pts = 'Beam-W_W8x1000_pulg_mb_368.pts'
#pts = 'Beam-W_W4x1301_pulg_mb_261.pts'
#pts = "Beam-W_W8x2102_pulg_ac_208.pts"
dpf=o3d.io.read_point_cloud("E:\\INNOVATE\\DB_TRAINING_1024_PTS\\"+pts.split('_')[0]+"\\"+pts)


ev1,ev2=project_pcl_pca(dpf,
             voxel_size=0.02,
             nb=5,
             std_ratio=1.5,
             p_th=0.1,
             ig="Linearity",
             upper=True
            )

o3d.visualization.draw_geometries([dpf])

In [95]:
ev1, ev2

(array([-0.8103288 ,  0.58596194,  0.00397953]),
 array([-0.17503539, -0.24852712,  0.95267879]))

In [96]:
aux1 = np.cross(ev1, np.array([0.0, 0.0, 1.0]))
aux1 = aux1/np.linalg.norm(aux1)
aux2 = ev1[2]/np.linalg.norm(ev1)
aux2 = np.arccos(aux2)
rot = return_rot(aux1, aux2)

In [97]:
cloud = dpf
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=80.0, origin=cloud.get_center())
cloud.rotate(rot)
o3d.visualization.draw_geometries([cloud, mesh_frame])

In [98]:
p = np.asarray(cloud.points)
n_p = np.copy(p)


low_bound = np.min(n_p[:, 2])
high_bound = np.max(n_p[:, 2])
l = high_bound - low_bound
N_SLICES = 10
nd = {i:0 for i in range(N_SLICES)}
pd = {i:[] for i in range(N_SLICES)}

for cont, i in enumerate(n_p):
    for j in range(N_SLICES):
        if i[2]>= low_bound+j*l/N_SLICES and i[2] < low_bound+(j+1)*l/N_SLICES:
            nd[j] += 1
            pd[j].append([n_p[cont, 0], n_p[cont, 1]])
            n_p[cont, 2] = low_bound+j*l/N_SLICES
print(nd)
test = o3d.geometry.PointCloud()
test.points = o3d.utility.Vector3dVector(n_p)
o3d.visualization.draw_geometries([test])

{0: 1598, 1: 4109, 2: 3143, 3: 4148, 4: 3399, 5: 3619, 6: 2769, 7: 2666, 8: 2817, 9: 1862}


In [102]:
ww = []
hh = []
for i in pd.keys():
    pd[i] = np.array(pd[i])
    co = pd[i].T.dot(pd[i])
    _, v = np.linalg.eig(co)
    aux = pd[i]
    init = np.random.random((aux.shape[0], 3))
    init[:, :2] = aux
    naux = o3d.geometry.PointCloud()
    naux.points = o3d.utility.Vector3dVector(init)
    naux.rotate(naux.get_rotation_matrix_from_axis_angle(np.array([0., 0., np.arctan(v[0, 1]/v[0, 0])])))
    puntex = np.asarray(naux.points)
    w = np.max(puntex[:, 1]) - np.min(puntex[:, 1])
    h = np.max(puntex[:, 0]) - np.min(puntex[:, 0])
    ww.append(w)
    hh.append(h)

    naux.translate(-naux.get_center())
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=8.0, origin=np.array([0., 0., 0.]))
    o3d.visualization.draw_geometries([naux, mesh_frame])
np.mean(np.array(ww))
np.mean(np.array(hh))

312.63235973272714

In [ ]:
np.cov(pd[0])]

In [35]:
co = pd[1].T.dot(pd[1])

In [36]:
w, v = np.linalg.eig(co)

In [37]:
v

array([[-0.92263623, -0.38567135],
       [ 0.38567135, -0.92263623]])

In [69]:

mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=80.0, origin=cloud.get_center())
mesh_frame.rotate(mesh_frame.get_rotation_matrix_from_axis_angle(np.array([0., 0., np.arctan(-0.385/0.922)])))
mesh_frame.translate(np.array([0, 0, 3*l/10]))
o3d.visualization.draw_geometries([mesh_frame, test])

In [30]:
np.arctan(-0.385, -0.922)

TypeError: return arrays must be of ArrayType

In [40]:
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=80.0, origin=cloud.get_center())
test.rotate(mesh_frame.get_rotation_matrix_from_axis_angle(np.array([0., 0., np.arctan(0.385/0.922)])))

o3d.visualization.draw_geometries([mesh_frame, test])